# Handwritten Digit Recognition with LeNet-5 and MNIST
In this simple playground we First use Fireball APIs to creare an MNIST dataset (https://en.wikipedia.org/wiki/MNIST_database) and a LeNet-5 (http://yann.lecun.com/exdb/lenet/) 
network. We then train the model using the training dataset and evaluate the model using
the test dataset.

## Create a MNIST dataset
Let's first load the MNIST dataset and review the dataset statistics.

In [1]:
import numpy as np
import time, os
from fireball import Model, myPrint
from fireball.datasets.mnist import MnistDSet
gpus = '0'

MnistDSet.download()

trainDs, validDs, testDs = MnistDSet.makeDatasets('train,valid,test', batchSize=128)
MnistDSet.printDsInfo(trainDs, validDs, testDs)
MnistDSet.printStats(trainDs, validDs, testDs)

Extracting "/Users/shahab/data/mnist/mnist.zip" ...
Deleting "/Users/shahab/data/mnist/mnist.zip" ...
MnistDSet Dataset Info:
    Number of Classes .............................. 10
    Dataset Location ............................... /Users/shahab/data/mnist/
    Number of Training Samples ..................... 54000
    Number of Test Samples ......................... 6000
    Number of Validation Samples ................... 10000
    Sample Shape ................................... (28, 28, 1)
    +-------+------------------+--------------------+---------------+
    | Class | Training Samples | Validation Samples | Test Samples  |
    +-------+------------------+--------------------+---------------+
    | 0     | 5331       9.87% | 980          9.80% | 592     9.87% |
    | 1     | 6068      11.24% | 1135        11.35% | 674    11.23% |
    | 2     | 5362       9.93% | 1032        10.32% | 596     9.93% |
    | 3     | 5518      10.22% | 1010        10.10% | 613    10.22% |
    | 4 

## Create a LeNet-5 Fireball model and train on MNIST
Now let's create a LeNet-5 fireball model using the text string ```layersInfo``` to specify network structure.
For a complete explanation of the available layers and their syntax, please refer to [the layers documentation](https://interdigitalinc.github.io/Fireball/html/source/layers.html).

In [2]:
# Here we define the LeNet-5 network which has 2 convolutional layers followed by 3 fully connected layers
layersInfo = ('IMG_S28_D1,' +                 # The input layer takes a 28x28 image of depth 1 (monochrome)
              'CONV_K5_O6_Ps:ReLU:MP_K2,' +   # Conv, Kernel size 5, 6 out channels, "same" padding, ReLU, Max pool
              'CONV_K5_O16_Pv:ReLU:MP_K2,' +  # Conv, Kernel size 5, 16 out channels, "valid" padding, ReLU, Max pool
              'FC_O120:ReLU,FC_O84:ReLU,FC_O10:None,' +   # 3 fully connected layers
              'CLASS_C10')                    # Output layer provides probabilities for each one of 10 classes

model = Model('LeNet-5', layersInfo = layersInfo,
              trainDs=trainDs, validationDs=validDs, # Train and validation datasets are given to the model
              optimizer = 'Adam',
              numEpochs = 5,
              learningRate = (0.01,0.001),    # Learning rate starts at 0.01 and exponentially decays to 0.001
              gpus=gpus)

model.printLayersInfo()
model.printNetConfig()
model.initSession()
model.train()


Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v               5 5 16        ReLU     MP(KSP):2 2 v    2,416      
L3_FC            5 5 16                                 120           ReLU                      48,120     
L4_FC            120                                    84            ReLU                      10,164     
L5_FC            84                                     10            None                      850        
OUT_CLASS        10            10 classes               10            None                      0          
---------------------------

## Evaluating the model
Now that we have a trained model, let's run inference on all test samples in the test dataset and compares the results with the ground-truth. The ```evaluateDSet``` function does exactly this.

In [3]:
results = model.evaluateDSet(testDs)

  Processed 10000 Sample. (Time: 0.48 Sec.)                              

Observed Accuracy: 0.991700
Expected Accuracy: 0.100354
Kappa: 0.990774 (Excellent)


## Save the model to the ```Models``` directory

In [4]:
model.save("Models/LeNet5.fbm")

## Where do I go from here?

[Reducing number of parameters of LeNet-5 Model](LeNet5-MNIST-Reduce.ipynb)

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)

[Quantizing the LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to CoreML](LeNet5-MNIST-CoreML.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

[Hand-written Didgit Recognition as a Regression problem](Regression.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)
